# 04/14/23

This notebook is intended to test chemformer loading. 

In [1]:
%load_ext autoreload
%autoreload 2

from models.chemformer.molbart import BARTModel
from models.chemformer.tokeniser import MolEncTokeniser
from models.chemformer.utils import REGEX, DEFAULT_MAX_SEQ_LEN
from models.chemformer.molbart_dataset import Uspto50
from models.chemformer.molbart_datamodule import FineTuneReactionDataModule
from models.chemformer.decoder import DecodeSampler

from datasets.dataset_utils import pad, tokenise_and_mask
from datasets.generic_index_dataset import GenericIndexedModule

from models.ranked_transformer import Moonshot

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the model. Using code from the original chemformer repo (```molbart/util.py```)

In [3]:
# Vocabulary
from pathlib import Path
vocab_path = "tempdata/chemformer/bart_vocab.txt"
chem_token_start = 272
tokeniser = MolEncTokeniser.from_vocab_file(
  vocab_path, REGEX, chem_token_start
)

In [3]:
tokeniser.tokenise(["COC(=O)CCC(=O)c1ccc(OC2CCCCO2)cc1O", "COOH"], pad=True)

{'original_pad_masks': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1]],
 'masked_pad_masks': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1]],
 'original_tokens': [['^',
   'C',
   'O',
   'C',
   '(',
   '=',
   'O',
   ')',
   'C

In [4]:
# Dataset
data_path = "tempdata/chemformer/uspto_50.pickle"
aug_prob = 0.0
dataset = Uspto50(
  data_path, aug_prob, forward=True
)


Uspto50 __init()__: 
[DS] <class 'pandas.core.frame.DataFrame'> 50037


In [5]:
# Model
model_type = "bart"
uni_model = model_type == "unified"
batch_size = 2
train_tokens = None
num_buckets = None

dm = FineTuneReactionDataModule(
    dataset,
    tokeniser,
    batch_size,
    DEFAULT_MAX_SEQ_LEN,
    forward_pred=True,
    val_idxs=dataset.val_idxs,
    test_idxs=dataset.test_idxs,
    train_token_batch_size=train_tokens,
    num_buckets=num_buckets,
    unified_model=uni_model
)
sampler = DecodeSampler(tokeniser, DEFAULT_MAX_SEQ_LEN)

Using a batch size of 2.
Building data module for forward prediction task...


### Chemformer Hacking

In [6]:
import torch
model_path = "tempdata/chemformer/model.ckpt"
obj = torch.load(model_path)

In [8]:
direct = Path("tempdata/SMILES_dataset")
features = ["HSQC", "SMILES"]
feature_handlers = [pad, tokenise_and_mask]
gim = GenericIndexedModule(direct, features, feature_handlers, len_override = 5, molbart_tokeniser = tokeniser)
gim.setup("fit")
train_dl = gim.train_dataloader()

In [19]:
model = Moonshot.load_from_checkpoint(
  model_path, strict=False, module_only=True, dim_model=512, dim_coords="224,224,64"
).cuda()

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:201: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.2.3 to v2.0.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file tempdata/chemformer/model.ckpt`


Pushed an encoder with no defined bounds
Pushed an encoder with no defined bounds
Pushed an encoder with no defined bounds


/opt/conda/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:153: UserWarning: Found keys that are in the model state dict but not in the checkpoint: ['latent', 'fc.weight', 'fc.bias', 'transformer_encoder.layers.0.self_attn.in_proj_weight', 'transformer_encoder.layers.0.self_attn.in_proj_bias', 'transformer_encoder.layers.0.self_attn.out_proj.weight', 'transformer_encoder.layers.0.self_attn.out_proj.bias', 'transformer_encoder.layers.0.linear1.weight', 'transformer_encoder.layers.0.linear1.bias', 'transformer_encoder.layers.0.linear2.weight', 'transformer_encoder.layers.0.linear2.bias', 'transformer_encoder.layers.0.norm1.weight', 'transformer_encoder.layers.0.norm1.bias', 'transformer_encoder.layers.0.norm2.weight', 'transformer_encoder.layers.0.norm2.bias', 'transformer_encoder.layers.1.self_attn.in_proj_weight', 'transformer_encoder.layers.1.self_attn.in_proj_bias', 'transformer_encoder.layers.1.self_attn.out_proj.weight', 'transformer_encoder.layers.1.self_attn.out_pro

In [24]:
trainer = Trainer(
    limit_test_batches=1,
    max_epochs=1, 
)
results = trainer.fit(model, datamodule=gim)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_test_batches=1)` was configured so 1 batch will be used.
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type               | Params
-----------------------------------------------------------
0 | enc                 | CoordinateEncoder  | 0     
1 | loss                | BCEWithLogitsLoss  | 0     
2 | fc                  | Linear             | 3.2 M 
3 | transformer_encoder | TransformerEncoder | 16.8 M
4 | emb                 | Embedding    

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]decoder_inputs.size()=torch.Size([5, 39])
decoder_mask.size()=torch.Size([5, 39])
tgt_mask.size()=torch.Size([39, 39])
decoder_embs.size()=torch.Size([39, 5, 512])
memory.size()=torch.Size([13, 5, 512])
encoder_mask.size()=torch.Size([5, 13])
Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s]                             decoder_inputs.size()=torch.Size([5, 41])
decoder_mask.size()=torch.Size([5, 41])
tgt_mask.size()=torch.Size([41, 41])
decoder_embs.size()=torch.Size([41, 5, 512])
memory.size()=torch.Size([22, 5, 512])
encoder_mask.size()=torch.Size([5, 22])
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s, v_num=39]decoder_inputs.size()=torch.Size([5, 39])
decoder_mask.size()=torch.Size([5, 39])
tgt_mask.size()=torch.Size([39, 39])
decoder_embs.size()=torch.Size([39, 5, 512])
memory.size()=torch.Size([13, 5, 512])
encoder_mask.size()=torch.Size([5, 13])
Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it, v_num=39]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:03<00:00,  3.41s/it, v_num=39]


In [ ]:
print(obj["state_dict"]["decoder.layers.1.norm1.bias"][:5])
print(model.decoder.layers[1].norm1.bias[:5])

tensor([ 0.0847, -0.0391,  0.0424, -0.0030, -0.0634], grad_fn=<SliceBackward0>)

### Continuing the normal code...

In [ ]:
DEFAULT_NUM_BEAMS = 10
model_path = "tempdata/chemformer/model.ckpt"
model = BARTModel.load_from_checkpoint(
  model_path,
  decode_sampler = sampler,
  my_tokeniser = tokeniser,
)
model.cuda().eval()
model.num_beams = DEFAULT_NUM_BEAMS

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:201: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.2.3 to v2.0.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file tempdata/chemformer/model.ckpt`


In [ ]:
logger = TensorBoardLogger("tb_logs", name=f"eval_bart_uspto_50")
trainer = Trainer(
    logger=logger,
    limit_test_batches=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_test_batches=1)` was configured so 1 batch will be used.


In [ ]:
results = trainer.test(model, datamodule=dm)
def print_results(results):
  print(f"Results for model: {model_path}")
  print(f"{'Item':<25}Result")
  for key, val in results.items():
    print(f"{key:<25} {val:.4f}")
print_results(results[0])

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
=== Describing forward input ===
==> Key: encoder_input
==> Value: <class 'torch.Tensor'>
==> Size: torch.Size([38, 2]) torch.cuda.LongTensor torch.int64
==> Decoded:  ['C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O', 'COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N']
==> value
	tensor([[  2,   2],
	        [272, 272],
	        [274, 285],
	        [280, 272],
	        [272, 275]], device='cuda:0')
==> Key: encoder_pad_mask
==> Value: <class 'torch.Tensor'>
==> Size: torch.Size([38, 2]) torch.cuda.BoolTensor torch.bool
==> value
	tensor([[False, False],
	        [False, False],
	        [False, False],
	        [False, False],
	        [False, False]], device='cuda:0')
==> Key: decoder_input
==> Value: <class 'torch.Tensor'>
==> Size: torch.Size([35, 2]) torch.cuda.LongTensor torch.int64
==> Decoded:  ['COC(=O)CCC(=O)c1ccc(OC2CCCCO2)cc1O', 'COC(=O)c1cccc(-c2nc3cccnc3[nH]2)c1']
==> value
	tensor([[  2,   2],
	        [272, 272],
	        [285, 285],